In [1]:
import os, sys
parent_dir = os.path.abspath('..')
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

In [43]:
import pandas as pd
import datetime
from pyecharts import options as opts
from pyecharts.charts import Line, Timeline, Bar
import os
from service.InfluxService import InfluxdbService
from utils.InfluxTime import InfluxTime

In [3]:
class GetData:
    def __init__(self):
        self.db = InfluxdbService()

    def get_data(self, start, end, measurement, targetcode=None, opcode=None, df=True, keep=None, filter_=None):
        return self.db.query_influx(start, end, measurement, targetcode=targetcode, opcode=opcode, df=df, keep=keep,
                                    filter_=filter_)

In [4]:
getData=GetData()

2023-02-15 13:03:54,946 - INFO - InfluxDB Connected


In [5]:
end=InfluxTime.utc8now()
print(end)

2023-02-15 13:03:54


In [117]:
df=getData.get_data(start="2023-02-10 00:00:00",
                    end=end,
                    measurement="optargetquote",targetcode="000016.XSHG", df=True)

In [122]:
df

,_time,_measurement,targetcode,pct,price
0,2023-02-10 09:30:00,optargetquote,000016.XSHG,-0.003092,2753.33
1,2023-02-10 09:31:00,optargetquote,000016.XSHG,-0.001191,2750.05
2,2023-02-10 09:32:00,optargetquote,000016.XSHG,-0.001204,2746.74
3,2023-02-10 09:33:00,optargetquote,000016.XSHG,0.000022,2746.80
4,2023-02-10 09:34:00,optargetquote,000016.XSHG,0.000098,2747.07
...,...,...,...,...,...
475,2023-02-13 14:55:00,optargetquote,000016.XSHG,0.000213,2767.91
476,2023-02-13 14:56:00,optargetquote,000016.XSHG,0.000231,2768.55
477,2023-02-13 14:57:00,optargetquote,000016.XSHG,-0.000011,2768.52
478,2023-02-13 14:58:00,optargetquote,000016.XSHG,0.000000,2768.52


In [119]:
def process_df(df)->dict:
    dic=df.to_dict('list')
    #dic["_measurement"]=dic["_measurement"][0]
    #dic["targetcode"]=dic["targetcode"][0]
    return dic

In [125]:
def DrawLine(df):
    if type(df)!=dict:
        df=process_df(df)
    line=Line()
    line.add_xaxis(xaxis_data=df["_time"])
    for i in df:
        line.add_yaxis(i,y_axis=df[i])
    
    line.set_series_opts(label_opts=opts.LabelOpts(is_show=False),)
    line.set_global_opts(title_opts=opts.TitleOpts(title=df["_measurement"][0].upper(),pos_right="50%"),
                    legend_opts=opts.LegendOpts(pos_right="10%",pos_top="20%",orient="vertical"),
                    tooltip_opts =opts.TooltipOpts(trigger='axis',axis_pointer_type='cross'),
                    toolbox_opts =opts.ToolboxOpts(is_show=True,orient='horizontal'),
                    datazoom_opts=opts.DataZoomOpts(range_start=95,range_end=100),
                    xaxis_opts=opts.AxisOpts(type_="time"),)
    
    return line

In [126]:
l=DrawLine(df)
l.render_notebook()